In [4]:
#imports!
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from scipy.stats import zscore
from google.colab import files

# Descaling function for Barrett's custom normalization
def descale_lat(scaled_lat):
    return (scaled_lat * 10) + 14

def descale_lon(scaled_lon):
    return (scaled_lon * 10) + 120

# Functions for anomaly detection, calculating distance, and normalizing z-score (for threat score)
def calculate_distance(lat, lon):
    return np.sqrt(lat**2 + lon**2)

def classify_anomaly(z):
    if abs(z) >= 3:
        return 'High'
    if abs(z) >= 2:
        return 'Medium'
    return 'Low'

def normalize_zscore(z):
    return (z - z.min()) / (z.max() - z.min())

# Upload and process user's data file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

# Redefining all points in avgLat and avgLon column to be the actual coordinate values
data['AvgLat'] = descale_lat(data['AvgLat'])
data['AvgLon'] = descale_lon(data['AvgLon'])

# Normalizing using MinMaxScaler
scaler = MinMaxScaler()
data[['AvgLat', 'AvgLon', 'AvgHead']] = scaler.fit_transform(data[['AvgLat', 'AvgLon', 'AvgHead']])
features = ['AvgLat', 'AvgLon', 'AvgHead']
target = ['AvgLat', 'AvgLon', 'AvgHead']

X = np.array(data[features]).reshape((data.shape[0], 1, len(features)))
y = np.array(data[target])

# Creating and compiling the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(len(target)))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])
history = model.fit(X, y, epochs=50, validation_split=0.2, verbose=1)

# Predicting values
y_pred = model.predict(X)

# Setting up loss functions - overkill :)
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Inversing the predicted and actual values to get the proper coordinate values again
y_pred_inverse = scaler.inverse_transform(y_pred)
y_actual_inverse = scaler.inverse_transform(y)

# Mapping predictions back
data['Pred_AvgLat'] = y_pred_inverse[:, 0]
data['Pred_AvgLon'] = y_pred_inverse[:, 1]
data['Pred_AvgHead'] = y_pred_inverse[:, 2]
data['Actual_AvgLat'] = y_actual_inverse[:, 0]
data['Actual_AvgLon'] = y_actual_inverse[:, 1]
data['Actual_AvgHead'] = y_actual_inverse[:, 2]
data['Actual_Distance'] = calculate_distance(data['Actual_AvgLat'], data['Actual_AvgLon'])
data['Pred_Distance'] = calculate_distance(data['Pred_AvgLat'], data['Pred_AvgLon'])
data['Zscore_Distance'] = zscore(data['Actual_Distance'] - data['Pred_Distance'])
data['Distance_Anomaly_Level'] = data['Zscore_Distance'].apply(classify_anomaly)

# Calculating z-scores for size and ICD and assigning anomaly level based on z-score
data['Zscore_Actual_Size'] = zscore(data['Size'])
data['Zscore_Actual_IntraClusterDistance'] = zscore(data['IntraClusterDistance'])
data['Size_Anomaly_Level'] = data['Zscore_Actual_Size'].apply(classify_anomaly)
data['IntraClusterDistance_Anomaly_Level'] = data['Zscore_Actual_IntraClusterDistance'].apply(classify_anomaly)

# Calculating a threat score by normalizing z-scores to be in the range [0, 1]
data['Size_Threat_Score'] = normalize_zscore(data['Zscore_Actual_Size'])
data['IntraCluster_Threat_Score'] = normalize_zscore(data['Zscore_Actual_IntraClusterDistance'])
data['Distance_Threat_Score'] = normalize_zscore(data['Zscore_Distance'])

# Calculating overall threat score taking the average of the individual threat scores
data['Overall_Threat_Score'] = (data['Size_Threat_Score'] + data['IntraCluster_Threat_Score'] + data['Distance_Threat_Score']) / 3

data.drop(columns=['AvgLat', 'AvgLon', 'AvgHead'], inplace=True)

# Saving and downloading CSV
output_file_name = 'predictions_w_normalized_threat_scores.csv'
data.to_csv(output_file_name, index=False)
files.download(output_file_name)
data.head()

Saving clusterdata_centroids (2).csv to clusterdata_centroids (2) (3).csv
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.1449 - mean_absolute_error: 0.2698 - val_loss: 0.0236 - val_mean_absolute_error: 0.1170
Epoch 2/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0062 - mean_absolute_error: 0.0466 - val_loss: 0.0014 - val_mean_absolute_error: 0.0239
Epoch 3/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 4.8289e-04 - mean_absolute_error: 0.0108 - val_loss: 4.3848e-04 - val_mean_absolute_error: 0.0134
Epoch 4/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 2.5075e-04 - mean_absolute_error: 0.0082 - val_loss: 3.1850e-04 - val_mean_absolute_error: 0.0110
Epoch 5/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.8164e-04 - mean_absolute_error: 0.0071 - val_loss: 2.5374e-04 - val_mean_absolute_error: 0.0104
Epoch 6/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.2604e-04 - mean_absolute_error: 0.0062 - val_loss: 1.9870e-04 - val_mean_absolute_error: 0.0083
Epoch 7/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 9.1021e-05 - mea

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Unnamed: 0,Cluster,Size,IntraClusterDistance,Hour,Pred_AvgLat,Pred_AvgLon,Pred_AvgHead,Actual_AvgLat,Actual_AvgLon,...,Zscore_Distance,Distance_Anomaly_Level,Zscore_Actual_Size,Zscore_Actual_IntraClusterDistance,Size_Anomaly_Level,IntraClusterDistance_Anomaly_Level,Size_Threat_Score,IntraCluster_Threat_Score,Distance_Threat_Score,Overall_Threat_Score
0,2747,3,2.0,0.032238,19200.0,23.294453,116.880287,25.917814,23.313805,116.892696,...,-0.970045,Low,-0.279253,0.033221,Low,Low,0.042553,0.196152,0.540438,0.259714
1,4579,2,2.0,0.090952,33708.0,23.240580,117.134415,29.161095,23.255915,117.146957,...,-0.996852,Low,-0.279253,1.938637,Low,Low,0.042553,0.553397,0.539128,0.378360
2,4435,2,2.0,0.032699,32808.0,20.938972,112.412041,44.389278,20.979667,112.454752,...,0.382398,Low,-0.279253,0.048192,Low,Low,0.042553,0.198959,0.606516,0.282676
3,2657,2,2.0,0.000344,18792.0,21.533587,111.827377,-1.153588,21.586326,111.821593,...,-1.439995,Low,-0.279253,-1.001803,Low,Low,0.042553,0.002095,0.517477,0.187375
4,2035,2,2.0,0.048766,15612.0,22.664234,115.823372,-0.882194,22.704982,115.871663,...,0.615402,Low,-0.279253,0.569579,Low,Low,0.042553,0.296713,0.617900,0.319056
